In [1]:
import pandas as pd
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer

import numpy as np

# Get the interactive Tools for Matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')


from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from numpy import dot
from numpy.linalg import norm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\harsh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
unable to import 'smart_open.gcs', disabling that module


In [2]:
from gensim import corpora
from gensim.models import TfidfModel
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
main_df = pd.read_csv('main_workshop.csv')
democrats_df1 =  pd.read_csv('democrats_workshop.csv')
republicans_df1 = pd.read_csv('republicans_workshop.csv')

main_df = main_df.dropna(subset = ['text']).reset_index(drop=True)
democrats_df1 = democrats_df1.dropna(subset = ['text']).reset_index(drop=True)
republicans_df1 = republicans_df1.dropna(subset = ['text']).reset_index(drop=True)

In [7]:
len(main_df), len(democrats_df1), len(republicans_df1)

(4705, 2445, 2260)

In [6]:
main_df = main_df.drop_duplicates()
main_df = main_df.reset_index(drop=True)

democrats_df1 = democrats_df1.drop_duplicates()
democrats_df1 = democrats_df1.reset_index(drop=True)

republicans_df1 = republicans_df1.drop_duplicates()
republicans_df1 = republicans_df1.reset_index(drop=True)

In [8]:
main_df

,title,body,politics,text
0,Trump campaign blames Democrats and RINOs for ...,,1,trump campaign blames democrats rinos brad par...
1,RIP to a legend (OC),,1,rip legend ( oc )
2,Trump campaign strategy to deter millions of B...,,1,trump campaign strategy deter millions black a...
3,Amy Coney Barrett suggested Obama’s nomination...,"""We're talking about Justice Scalia, the staun...",1,amy coney barrett suggested obama ’ nomination...
4,American Fascism: A German Writer's Urgent War...,,1,american fascism : german writer 's urgent war...
...,...,...,...,...
4700,Wife of Boston Marathon hero said Biden touche...,,0,wife boston marathon hero said biden touched '...
4701,Can’t imagine!,,0,’ imagine !
4702,Americans Come Out Of Debate Excited By Their ...,,0,americans come debate excited two great choices
4703,Biden: 'You Have To Elect Me To Find Out What ...,,0,biden : 'you elect find policy positions '


In [9]:
import re
def cutom_tokenizer(str):
    s = str.lower()
    s = re.split('https:\/\/.*|http:\/\/.*',s)
    s = re.split('pic.tw*',s[0])
    tokens = nltk.word_tokenize(s[0])
    remove_stopwords = list(filter(lambda token: token not in stopwords.words("english"),tokens))
    tokenize_words = [word for word in remove_stopwords]
    return tokenize_words

In [10]:
main_df['text'] = main_df.apply(lambda row: ' '.join(cutom_tokenizer(row['text'])), axis=1)
democrats_df1['text'] = democrats_df1.apply(lambda row: ' '.join(cutom_tokenizer(row['text'])), axis=1)
republicans_df1['text'] = republicans_df1.apply(lambda row: ' '.join(cutom_tokenizer(row['text'])), axis=1)


# create a main dataframe
#main_df = pd.concat([main_left_df,main_right_df,center_data],ignore_index=True)

In [103]:
main_df = main_df.drop_duplicates()
main_df = main_df.reset_index(drop=True)

democrats_df1 = democrats_df1.drop_duplicates()
democrats_df1 = democrats_df1.reset_index(drop=True)

republicans_df1 = republicans_df1.drop_duplicates()
republicans_df1 = republicans_df1.reset_index(drop=True)

In [104]:
len(main_df), len(democrats_df1), len(republicans_df1)

(72439, 39675, 17077)

In [11]:
main_df = main_df.sample(frac=1).reset_index(drop=True)

In [12]:
main_df.to_csv('main_df_1.csv')
democrats_df1.to_csv('democrats_df1_1.csv')
republicans_df1.to_csv('republicans_df1_1.csv')


In [13]:
main_df

,title,body,politics,text
0,Turns Out California Democrats Used Illegals F...,,0,turns california democrats used illegals ballo...
1,Florida Senate approves bill allowing armed te...,,0,florida senate approves bill allowing armed te...
2,What to expect on November 3rd and the state o...,,1,expect november 3rd state race .
3,What’s my name again,,0,’ name
4,Trump fires back after Dems indicate impeachme...,,0,trump fires back dems indicate impeachment cou...
...,...,...,...,...
4700,So sad. RIP.,,0,sad . rip .
4701,The tolerant Left,,0,tolerant left
4702,Virginia will eliminate a state holiday honori...,,1,virginia eliminate state holiday honoring robe...
4703,"Opinion | To Beat Trump, Mock Him",,1,"opinion | beat trump , mock"


In [14]:
dictionary = corpora.Dictionary([i.split(' ') for i in main_df['text']]) # using genism library 

In [15]:
dictionary.dfs

{5: 11,
 1: 47,
 2: 252,
 6: 33,
 4: 4,
 0: 50,
 3: 10,
 11: 46,
 12: 176,
 8: 4,
 10: 44,
 7: 12,
 9: 12,
 13: 6,
 16: 15,
 17: 43,
 15: 11,
 19: 104,
 18: 65,
 14: 1446,
 21: 848,
 20: 24,
 33: 1248,
 29: 13,
 24: 85,
 28: 53,
 31: 5,
 30: 25,
 26: 124,
 25: 11,
 27: 180,
 32: 38,
 23: 1020,
 35: 216,
 22: 1309,
 34: 71,
 39: 1,
 44: 19,
 49: 19,
 48: 92,
 46: 107,
 41: 64,
 38: 566,
 40: 54,
 43: 29,
 37: 1,
 42: 32,
 36: 343,
 47: 218,
 45: 72,
 51: 4,
 54: 2,
 50: 7,
 52: 4,
 53: 37,
 60: 124,
 56: 348,
 63: 1,
 58: 22,
 64: 212,
 61: 18,
 59: 82,
 55: 471,
 62: 119,
 57: 3,
 65: 39,
 66: 177,
 68: 4,
 71: 5,
 72: 19,
 70: 26,
 67: 9,
 69: 42,
 73: 10,
 78: 1,
 76: 1,
 75: 24,
 77: 266,
 74: 2,
 80: 3,
 86: 1,
 81: 23,
 79: 1,
 84: 114,
 82: 4,
 85: 15,
 87: 146,
 83: 132,
 88: 4,
 99: 21,
 89: 215,
 90: 221,
 92: 40,
 94: 29,
 91: 174,
 97: 1,
 95: 3,
 100: 1,
 96: 3,
 93: 15,
 98: 21,
 103: 194,
 104: 6,
 101: 1,
 105: 7,
 102: 8,
 106: 5,
 108: 25,
 109: 36,
 117: 4,
 116: 8,
 

In [16]:
dictionary.cfs

{5: 11,
 1: 58,
 2: 303,
 6: 47,
 4: 4,
 0: 62,
 3: 10,
 11: 54,
 12: 248,
 8: 4,
 10: 46,
 7: 14,
 9: 17,
 13: 6,
 16: 15,
 17: 43,
 15: 14,
 19: 157,
 18: 93,
 14: 4340,
 21: 1466,
 20: 30,
 33: 1581,
 29: 14,
 24: 93,
 28: 61,
 31: 5,
 30: 25,
 26: 160,
 25: 11,
 27: 254,
 32: 39,
 23: 1576,
 35: 246,
 22: 3830,
 34: 90,
 39: 1,
 44: 24,
 49: 20,
 48: 118,
 46: 142,
 41: 67,
 38: 851,
 40: 58,
 43: 32,
 37: 1,
 42: 38,
 36: 421,
 47: 337,
 45: 77,
 51: 4,
 54: 2,
 50: 8,
 52: 4,
 53: 37,
 60: 139,
 56: 1030,
 63: 1,
 58: 25,
 64: 229,
 61: 19,
 59: 86,
 55: 802,
 62: 151,
 57: 3,
 65: 47,
 66: 182,
 68: 4,
 71: 5,
 72: 19,
 70: 36,
 67: 9,
 69: 55,
 73: 11,
 78: 1,
 76: 1,
 75: 24,
 77: 361,
 74: 2,
 80: 3,
 86: 1,
 81: 26,
 79: 1,
 84: 125,
 82: 4,
 85: 18,
 87: 193,
 83: 150,
 88: 4,
 99: 25,
 89: 614,
 90: 629,
 92: 51,
 94: 29,
 91: 277,
 97: 1,
 95: 3,
 100: 1,
 96: 3,
 93: 15,
 98: 22,
 103: 247,
 104: 6,
 101: 1,
 105: 9,
 102: 8,
 106: 5,
 108: 25,
 109: 45,
 117: 4,
 116: 9

In [17]:
dictionary.num_docs # total documens processed

4705

In [18]:
dictionary.token2id

{'ballot': 0,
 'california': 1,
 'democrats': 2,
 'harvesting': 3,
 'illegals': 4,
 'turns': 5,
 'used': 6,
 'allowing': 7,
 'approves': 8,
 'armed': 9,
 'bill': 10,
 'florida': 11,
 'senate': 12,
 'teachers': 13,
 '.': 14,
 '3rd': 15,
 'expect': 16,
 'november': 17,
 'race': 18,
 'state': 19,
 'name': 20,
 '’': 21,
 ',': 22,
 ':': 23,
 'back': 24,
 'block': 25,
 'could': 26,
 'court': 27,
 'dems': 28,
 'fires': 29,
 'impeachment': 30,
 'indicate': 31,
 'nominee': 32,
 'trump': 33,
 'win': 34,
 '‘': 35,
 "'": 36,
 "'more": 37,
 "'s": 38,
 '57': 39,
 'death': 40,
 'ginsburg': 41,
 'important': 42,
 'makes': 43,
 'percent': 44,
 'poll': 45,
 'say': 46,
 'vote': 47,
 'voters': 48,
 'young': 49,
 '21': 50,
 'bet': 51,
 'clips': 52,
 'second': 53,
 'shoots': 54,
 '?': 55,
 '``': 56,
 'alert': 57,
 'blame': 58,
 'covid-19': 59,
 'donald': 60,
 'families': 61,
 'first': 62,
 'jeepers-creepers': 63,
 'joe': 64,
 'public': 65,
 'says': 66,
 'amid': 67,
 'disapproval': 68,
 'government': 69,
 'h

In [20]:
dems_dictionary = corpora.Dictionary([i.split(' ') for i in democrats_df1['text']])

reps_dictionary = corpora.Dictionary([i.split(' ') for i in republicans_df1['text']])

In [21]:
tmp_fname = get_tmpfile("2023_dictionary")
dictionary.save('2023_dictionary.dict')
dems_dictionary.save('2023_dems_dictionary.dict')
reps_dictionary.save('2023_reps_dictionary.dict')